In [ ]:
from __future__ import annotations

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn

from scipy.integrate import solve_ivp
from scipy.integrate import odeint

import scipy

import sys
import gymnasium as gym
import time
import signal

# 定义环境的相关设置
balance_time = 240
h_in = 1 / 100

# 注册和创建自定义环境
CartPoleSwingUp = gym.register(
    id = 'CartPoleSwingUp',
    entry_point = 'myCartpoleF_SwingUp:CartPoleSwingUp', 
    # reward_threshold = -40 * 0.95,
    max_episode_steps = 10000,                                                  #int(balance_time / h_in)
)
env = gym.make('CartPoleSwingUp', render_mode='human')
print(gym.spec('CartPoleSwingUp'))

创建 NormalActionNoise 实例，用于在智能体的动作中加入噪声。具体的参数含义如下：

mean = np.zeros(env.action_space.shape): 这里的 mean 是噪声的均值，np.zeros(env.action_space.shape) 表示均值为零，且它的维度与环境的动作空间 env.action_space.shape 相同。env.action_space.shape 表示动作空间的维度。

sigma = 0.1 * np.ones(env.action_space.shape): 这里的 sigma 是噪声的标准差，0.1 * np.ones(env.action_space.shape) 表示标准差是 0.1，并且与动作空间的维度相同。这样每个维度的噪声标准差都是 0.1。

通过这种方式，动作中会加入一定的噪声，从而使得训练过程更加稳定，防止模型陷入局部最优解，增强探索性。

创建 DDPG 模型：

'MlpPolicy': 选择了多层感知器（MLP）作为策略网络的结构。该网络用于从状态中输出动作。

env: 环境对象，智能体将在该环境中进行训练。

policy_kwargs = dict(net_arch=[400, 300]): 这个参数定义了策略网络的结构，net_arch=[400, 300] 表示策略网络有两层隐藏层，分别有 400 个神经元和 300 个神经元。policy_kwargs 是传递给策略网络构造函数的附加参数。

verbose = 1: 这个参数设置了训练时的输出级别，verbose=1 表示打印训练过程中的信息。

tensorboard_log = "./ddpg_cartpole/": 这个参数指定了 TensorBoard 日志的存储路径，用于后期可视化训练过程。

action_noise=action_noise: 将前面创建的噪声对象 action_noise 传递给 DDPG 模型，用于在选择动作时加入噪声。

In [2]:
import gym
import numpy as np

from stable_baselines3.common.callbacks import BaseCallback

class RewardHistoryCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(RewardHistoryCallback, self).__init__(verbose)
        self.reward_list = []  # 用于存储每个 episode 的累计奖励

    def _on_step(self):
        # 获取当前 episode 的奖励
        if self.n_calls % self.model.n_envs == 0:  # 每个 episode 结束时
            self.reward_list.append(self.locals['rewards'].mean())
        return True

    def get_rewards(self):
        return self.reward_list


In [ ]:
# 步骤三：使用 DDPG 算法训练代理
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
# 将环境包装为向量环境
# env = DummyVecEnv([lambda: gym.make('CartPoleSwingUp', render_mode='human')])

# 初始化 DDPG 代理
from stable_baselines3.common.noise import NormalActionNoise
# 加入噪声

model = DDPG('MlpPolicy', env, policy_kwargs = dict(net_arch=[256, 128, 64]), verbose = 1)

reward_callback = RewardHistoryCallback()
model.learn(total_timesteps = 100000, callback=reward_callback)



In [ ]:
import matplotlib.pyplot as plt
# 获取奖励历史
reward_history = reward_callback.get_rewards()

# 平滑奖励（可选）
def smooth_rewards(rewards, window_size=10):
    return np.convolve(rewards, np.ones(window_size) / window_size, mode='valid')

smoothed_rewards = smooth_rewards(reward_history, window_size=10)

# 绘制奖励曲线
plt.plot(smoothed_rewards)
plt.xlabel('Episode')
plt.ylabel('Smoothed Reward')
plt.title('DDPG Training Reward Curve')
plt.grid(True)
plt.show()


In [5]:
model.save("ddpg_cartpole_model3")

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

import gym
import numpy as np

# 将环境包装为向量环境
# env = DummyVecEnv([lambda: gym.make('CartPoleSwingUp', render_mode='human')])

# 初始化 PPO 代理
model = PPO('MlpPolicy', env, policy_kwargs=dict(net_arch=[256, 128, 64]), verbose = 1)

# 训练模型
model.learn(total_timesteps = 500000)

# 保存训练后的模型
model.save("ppo_cartpole_model")


In [ ]:
model.save("ppo_cartpole_model")

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
import numpy as np

# 将环境包装为向量环境

# 初始化 SAC 代理
model = SAC('MlpPolicy', env, policy_kwargs=dict(net_arch=[256, 128, 64]), verbose=1, learning_rate=0.001)

# 训练模型
model.learn(total_timesteps=30000)

# 保存训练后的模型
model.save("sac_cartpole_model")


In [ ]:
from stable_baselines3 import TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.td3.policies import TD3Policy
import gym
import numpy as np

# 初始化 TD3 代理
model = TD3('MlpPolicy', env, policy_kwargs=dict(net_arch=[256, 128, 64]), verbose = 1)

# 训练模型
model.learn(total_timesteps = 200000)

# 保存训练后的模型
model.save("td3_cartpole_model")


In [ ]:
from stable_baselines3 import DDPG
model = DDPG.load("ddpg_cartpole_model3", env = env)
reward_callback = RewardHistoryCallback()
model.learn(total_timesteps = 100000, callback=reward_callback)

In [ ]:
import matplotlib.pyplot as plt
# 获取奖励历史
reward_history = reward_callback.get_rewards()

# 平滑奖励（可选）
def smooth_rewards(rewards, window_size=10):
    return np.convolve(rewards, np.ones(window_size) / window_size, mode='valid')

smoothed_rewards = smooth_rewards(reward_history, window_size=10)

# 绘制奖励曲线
plt.plot(smoothed_rewards)
plt.xlabel('Episode')
plt.ylabel('Smoothed Reward')
plt.title('DDPG Training Reward Curve')
plt.grid(True)
plt.show()


In [9]:
model.save("ddpg_cartpole_model3")

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DDPG
model = DDPG.load("ddpg_cartpole_model")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes = 10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DDPG
model = DDPG.load("ddpg_cartpole_model2")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes = 10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DDPG
model = DDPG.load("ddpg_cartpole_model3")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes = 10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselinecs3 import PPO
model = PPO.load("ppo_cartpole_model")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes = 10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")
